In [5]:
#Let's train some models on this data

In [6]:
import pandas as pd
import numpy as np
import os
import sys 
sys.path.append(os.path.dirname(os.getcwd()))
import warnings
import config as cfg
from sklearn.linear_model import LogisticRegression #importing for Logistic regression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

warnings.filterwarnings("ignore")


In [7]:
#importing libraries
data_df=pd.read_csv(cfg.input+"pre_processed.csv",index_col=0)
data_df.head(4)

,FCVC,NCP,CH2O,FAF,TUE,BMI,Target,Gender_Male,family_history_with_overweight_yes,FAVC_yes,...,CAEC_no,SMOKE_yes,SCC_yes,CALC_Sometimes,CALC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,log_age
0,2.000000,2.983297,2.763573,0.000000,0.976473,28.259565,6,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.196344
1,2.000000,3.000000,2.000000,1.000000,1.000000,23.422091,1,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.890372
2,1.880534,1.411685,1.910378,0.866045,1.673584,17.126706,0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.890372
3,3.000000,3.000000,1.674061,1.467863,0.780199,44.855798,4,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.042269


In [27]:
#splitting the data
scale=StandardScaler()
X,Y=data_df.drop('Target',axis=1),data_df['Target']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

Index(['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'BMI', 'Gender_Male',
       'family_history_with_overweight_yes', 'FAVC_yes', 'CAEC_Frequently',
       'CAEC_Sometimes', 'CAEC_no', 'SMOKE_yes', 'SCC_yes', 'CALC_Sometimes',
       'CALC_no', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'log_age'],
      dtype='object')


In [9]:
#Creating a random model for baseline value
def random_pred(df):
    return random.choice([0,1,2,3,4,5,6])
Y_rand_pred=pd.DataFrame(X_train).apply(random_pred,axis=1)

In [10]:
#Let's see the accuracy of random model
acc_rand=accuracy_score(Y_train,Y_rand_pred)
print("Accuracy of Random predictory:",acc_rand)

Accuracy of Random predictory: 0.14530892448512586


* So now we know that our models needs to outperform atleast this score

In [11]:
log_reg=LogisticRegression(multi_class='ovr')
log_reg.fit(X_train,Y_train)
Y_train_log=log_reg.predict(X_train)
Y_test_log=log_reg.predict(X_test)
acc_log_train=accuracy_score(Y_train,Y_train_log)
acc_log_test=accuracy_score(Y_test,Y_test_log)
print("Accuracy of logistic regression on train set:",acc_log_train)
print("Accuracy of logistic regression on testing set:",acc_log_test)

Accuracy of logistic regression on train set: 0.7646633746838493
Accuracy of logistic regression on testing set: 0.7639691714836223


* Well! Not Bad!!

In [12]:
#Training it on SVC
svc=OneVsRestClassifier(SVC()).fit(X_train,Y_train)
Y_train_svc=svc.predict(X_train)
Y_test_svc=svc.predict(X_test)
acc_svc_train=accuracy_score(Y_train,Y_train_svc)
acc_svc_test=accuracy_score(Y_test,Y_test_svc)
print("Accuracy of SVC on train set:",acc_svc_train)
print("Accuracy of SVC on test set:",acc_svc_test)

Accuracy of SVC on train set: 0.8365650969529086
Accuracy of SVC on test set: 0.7945568400770713


* Yo! This seems to be even better

In [13]:
rf=RandomForestClassifier(n_estimators=10,random_state=0,oob_score=True)
rf.fit(X_train,Y_train)
Y_train_rf=rf.predict(X_train)
Y_test_rf=rf.predict(X_test)
acc_rf_train=accuracy_score(Y_train,Y_train_rf)
acc_rf_test=accuracy_score(Y_test,Y_test_rf)
print("Accuracy of RF on train set:",acc_rf_train)
print("Accuracy of RF on test set:",acc_rf_test)

Accuracy of RF on train set: 0.9939178610140913
Accuracy of RF on test set: 0.8607899807321773


In [14]:
Y.value_counts()

Target
4    4046
3    3248
1    3082
2    2910
0    2523
6    2522
5    2427
Name: count, dtype: int64

In [15]:
len(X_train[0])

21

#### Clearly this is overfitting

#### How about using Boosting techniques: Let's see for XGBoost

In [16]:
xgb_clf=xgb.XGBClassifier(objective="multi:softmax",max_depth=3,n_estimators=60).fit(X_train,Y_train)
Y_train_xgb=xgb_clf.predict(X_train)
Y_test_xgb=xgb_clf.predict(X_test)
acc_xgb_train=accuracy_score(Y_train,Y_train_xgb)
acc_xgb_test=accuracy_score(Y_test,Y_test_xgb)
print("Accuracy of XGB on train set:",acc_xgb_train)
print("Accuracy of XGB on test set:",acc_xgb_test)

Accuracy of XGB on train set: 0.9007587618932916
Accuracy of XGB on test set: 0.8819845857418112


#### Woah! it improved

In [17]:
### Time to call mighty ANN
## Defining ann

ann=Sequential()
ann.add(Dense(units=21,activation='relu'))
ann.add(Dense(units=21,activation='relu'))
ann.add(Dense(units=7,activation='softmax'))

#Compile the trained model
ann.compile(optimizer='rmsprop',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
ann.fit(x=X_train,y=Y_train,epochs=50,batch_size=50,validation_data=(X_test,Y_test))




Epoch 1/50


333/333 [==============================] - 5s 7ms/step - loss: 1.2116 - accuracy: 0.5536 - val_loss: 0.8644 - val_accuracy: 0.6597
Epoch 2/50
333/333 [==============================] - 2s 6ms/step - loss: 0.7177 - accuracy: 0.7241 - val_loss: 0.6378 - val_accuracy: 0.7642
Epoch 3/50
333/333 [==============================] - 2s 5ms/step - loss: 0.5658 - accuracy: 0.7909 - val_loss: 0.5459 - val_accuracy: 0.8011
Epoch 4/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4946 - accuracy: 0.8224 - val_loss: 0.4988 - val_accuracy: 0.8227
Epoch 5/50
333/333 [==============================] - 2s 6ms/step - loss: 0.4585 - accuracy: 0.8374 - val_loss: 0.4829 - val_accuracy: 0.8326
Epoch 6/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4394 - accuracy: 0.8458 - val_loss: 0.4668 - val_accuracy: 0.8374
Epoch 7/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4286 - accuracy: 0.8484 - val_loss: 0.4643 - val_accuracy: 0.8403
Ep

In [18]:
## Let's try with LightGBM
bgm=lgb.LGBMClassifier()
bgm.fit(X_train,Y_train)
Y_train_bgm=bgm.predict(X_train)
Y_test_bgm=bgm.predict(X_test)
acc_bgm_train=accuracy_score(Y_train,Y_train_bgm)
acc_bgm_test=accuracy_score(Y_test,Y_test_bgm)
print("Accuracy of LightGBM on train set:",acc_bgm_train)
print("Accuracy of LightGBM on test set:",acc_bgm_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1786
[LightGBM] [Info] Number of data points in the train set: 16606, number of used features: 21
[LightGBM] [Info] Start training from score -2.114619
[LightGBM] [Info] Start training from score -1.916947
[LightGBM] [Info] Start training from score -1.957479
[LightGBM] [Info] Start training from score -1.865469
[LightGBM] [Info] Start training from score -1.621311
[LightGBM] [Info] Start training from score -2.134272
[LightGBM] [Info] Start training from score -2.118118
Accuracy of LightGBM on train set: 0.96820426351921
Accuracy of LightGBM on test set: 0.8795761078998073


In [19]:
#let's dump this models
joblib.dump(xgb_clf,cfg.models+'xgb_clf.joblib')
joblib.dump(ann,cfg.models+'ann.joblib')
joblib.dump(bgm,cfg.models+'bgm.joblib')
joblib.dump(scale,cfg.models+'scaler.joblib')

['../models/scaler.joblib']